<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 提示模板

In [ ]:
from videos.walkthroughs import walkthrough_16 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将学习如何用提示模板构建可重用的 LLM 功能，并开始使用 LangChain 提供的强大提示模板工具。

---

## 目标

当您完成这个 notebook 时，您将：

- 借助提示模板利用 LLM 能力的必要性。
- 能够使用 LangChain 创建可复用的提示模板。
- 利用提示模板对一系列提供的文本样本执行各种 LLM 驱动的任务。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 一次性任务 vs. 可复用功能

如果您是 LLM 应用的终端用户，特别是像 Perplexity 或 ChatGPT 这样的聊天机器人，您可能会经历一个迭代的提示词开发过程，以便从基于 LLM 的应用中获取对您有帮助的响应。然而，如果您是在为自己构建的 LLM 应用设计提示词，通常会想要开发一个针对某个任务或功能的提示词，并且能复用到多种输入中。

作为开发者，您已经非常熟悉从一次性任务到更通用、模板化功能的迭代方式。您可以通过很多方式做到这一点，现在来考虑一个简单且通用的例子，计算两个数字的乘积。

如果您只是想为自己进行一次性计算，您可以简单地写一行代码，如下所示：

In [ ]:
99 * 64

不过，如果您想获得更通用的功能，比如在多种输入中复用乘法功能，就可以写如下的函数：

In [ ]:
def multiply_two_numbers(a, b):
    return a * b

这样，就不仅能够进行一次计算，...

In [ ]:
multiply_two_numbers(99, 64)

...还可以将这个函数复用于任意数量的计算，包括对其他用户有用的计算。

---

## 提示模板作为可复用功能

提示词的工作原理也差不多。如果您有一个一次性任务，您只需为它写一个提示词：

In [ ]:
one_off_prompt = "Translate the following from English to Spanish: 'Today is a good day.'"

In [ ]:
print(llm.invoke(one_off_prompt).content)

如果您想创建可复用的功能，可以将提示词的一部分抽象为参数，这样就可以通过任意输入复用，如下所示：

In [ ]:
def translate_from_english_to_spanish(english_statement):
    return f"Translate the following from English to Spanish. Provide just the translated text: {english_statement}"

In [ ]:
english_statements = [
    'Today is a good day.',
    'Tomorrow will be even better.',
    'Next week, who can say.'
]

In [ ]:
prompts = [translate_from_english_to_spanish(english_statement) for english_statement in english_statements]

In [ ]:
prompts

In [ ]:
translations = llm.batch(prompts)

In [ ]:
for translation in translations:
    print(translation.content)

`translate_from_english_to_spanish` 函数创建了一个**提示模板**，它构建了将英文句子翻译成西班牙语的功能。

当然，如果我们愿意，还可以进一步抽象出更多内容，创建一个更通用的模板：

In [ ]:
def translate(from_language, to_language, statement):
    return f"Translate the following from {from_language} to {to_language}. Provide only the translated text: {statement}"

In [ ]:
print(llm.invoke(translate('English', 'French', 'Computers have many languages of their own')).content)

---

## LangChain 的 `ChatPromptTemplate.from_template`

开发自己的提示模板没什么问题，但 LangChain 提供了一个非常庞大的模板机制集合，易于使用、灵活、维护良好且被广泛应用。

我们将从创建聊天模型提示模板最基本的方式开始，即 `ChatPromptTemplate.from_template`。首先，我们需要将 `ChatPromptTemplate` 导入到我们的环境中。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

现在，我们可以创建一个模板，和之前通过函数创建的方式很类似。让我们再创建一个将英文句子翻译成西班牙语的模板。

In [ ]:
english_to_spanish_template = ChatPromptTemplate.from_template("""Translate the following from English to Spanish. \
Provide only the translated text: '{english_statement}'""")

如您所见，这和我们在 `translate_from_english_to_spanish` 函数中返回的 f-string 基本上是一样的。

要从模板创建实际的提示词，我们使用模板的 `invoke` 方法。

In [ ]:
prompt = english_to_spanish_template.invoke("Today is a good day.")

此时，我们可以像之前那样将其传递给我们的 LLM。

In [ ]:
print(llm.invoke(prompt).content)

---

## 聊天提示模板细节

如果我们仔细看看刚从模板创建的提示词，会发现这里不仅仅只是创建了一个字符串：

In [ ]:
print(prompt)

在背后，LangChain 似乎正在构建一个 `messages` 列表，其中 `HumanMessage` 中的 `content` 是我们打算创建的字符串提示词。

在整个课程中，您会学到很多关于 `messages` 的内容，包括 `HumanMessage`，但现在请回想一下之前的 notebook，当您开始与我们的本地 LLM 交互时，需要小心地使用 `OpenAI` 库与**聊天**补全入口交互。因为我们在使用聊天模型，并且在构建提示词时，必须为提示词添加一些额外的结构，包括创建 `messages` 列表并将提示词指定为与 `user` 角色相关。以下是之前 notebook 的内容：

```python
response = client.chat.completions.create(
    model=model,
    messages=[{'role': 'user', 'content': prompt}]
)
```

您现在需要理解的重点是，当专门处理聊天模型时，模型期望通过消息以轮询结构进行交互，每条消息会与特定角色相关，比如 AI 助手、人类用户或其他角色。

使用 LangChain 的一个好处是，很多与聊天模型的期望相符的特定格式要求都处理好了，但同时，您也可以在需要时控制程序。

---

## 多值的提示模板

作为参考，这里是我们之前创建的模板，用于从英语翻译成西班牙语。

In [ ]:
english_to_spanish_template = ChatPromptTemplate.from_template("""Translate the following from English to Spanish. \
provide only the translated text '{english_statement}'""")

prompt = english_to_spanish_template.invoke("Today is a good day.")

print(llm.invoke(prompt).content)

您可能注意到，在模板调用 `invoke` 时，我们传入了一个字符串...

In [ ]:
english_to_spanish_template.invoke("Today is a good day.")

...在这种情况下是可以的，因为模板 `"Translate the following from English to Spanish: '{english_statement}'"` 只期望一个值，即 `english_statement`。

当调用期望多个值的模板时，实际上即使是期望单个值的模板，最佳实践也是传入一个 `dict`，将模板占位符映射到它们的预期值。因此，调用我们模板的更好方式是：

In [ ]:
english_to_spanish_template.invoke({"english_statement": "Today is a good day."})

...如您所见，我们得到了相同的提示词。

在处理期望多个值的提示词时，通过字典项指定哪个字符串对应哪个占位符变得至关重要。在这里，我们演示了这个更一般化模板的创建和使用，该模板允许进行任意语言之间的翻译。

In [ ]:
translate_template = ChatPromptTemplate.from_template("Translate the following from {from_language} to {to_language}. \
proivde only the translated text: {statement}")

In [ ]:
prompt = translate_template.invoke({
    "from_language": "English",
    "to_language": "French",
    "statement": "Sometimes a little additional complexity is worth it."
})

In [ ]:
print(llm.invoke(prompt).content)

---

## 练习：创建提示模板

这个练习比之前的练习稍长。您将把 3 个与 LLM 相关的任务囊括到提示模板中，并将其应用于我们提供的语句列表。

您应该为以下与 LLM 相关的任务创建模板：
- 情感分析：确定给定文本的整体情感是“积极”还是“消极”。
- 主题识别：识别并阐明给定文本的首要主题。
- 后续问题生成：生成一个恰当且有趣的后续问题，以澄清给定文本的某些方面。

请使用下面的 `statements` 作为您每个模板所使用的文本。在完成后，您应该能够对所有 5 个提供的文本进行 3 个与 LLM 相关的任务。

In [ ]:
statements = [
    "I had a fantastic time hiking up the mountain yesterday.",
    "The new restaurant downtown serves delicious vegetarian dishes.",
    "I am feeling quite stressed about the upcoming project deadline.",
    "Watching the sunset at the beach was a calming experience.",
    "I recently started reading a fascinating book about space exploration."
]

如果您准备好迎接挑战，请随时开始您的工作。如果需要帮助，请点击下面的*逐步指导*展开练习指导。

### 您的代码

---

## 逐步指导

### 情感分析的提示模板

首先为每个 LLM 任务构建提示模板，从情感分析任务开始。

如果您卡住了，可以随时查看下面的*参考答案*。

### 您的代码

In [ ]:
sentiment_template = 'TODO'

### 参考答案

In [ ]:
sentiment_template = ChatPromptTemplate.from_template("""In a single word, either 'positive' or 'negative', \
provide the overall sentiment of the following piece of text: {text}""")

### 主题识别的提示模板

接下来为主题识别创建提示模板。

如果您卡住了，可以随时查看下面的*参考答案*。

### 您的代码

In [ ]:
main_topic_template = 'TODO'

### 参考答案

In [ ]:
main_topic_template = ChatPromptTemplate.from_template("""Identify and state, as concisely as possible, the main topic \
of the following piece of text. Only provide the main topic and no other helpful comments. Text: {text}""")

### 后续问题生成的提示模板

接下来创建一个用于生成相关后续问题的提示模板。

如果您卡住了，可以随时查看下面的*参考答案*。

### 您的代码

In [ ]:
followup_template = 'TODO'

### 参考答案

In [ ]:
followup_template = ChatPromptTemplate.from_template("""What is an appropriate and interesting followup question that would help \
me learn more about the provided text? Only supply the question. Text: {text}""")

### 创建批处理提示列表

为了给这 3 个 LLM 任务生成批量响应，我们需要为每个任务准备一个提示词列表。

在这个步骤中，结合上面定义的 `statements` 和您刚创建的每个提示模板，为 3 个 LLM 任务创建一个提示词列表。

如果您卡住了，可以随时查看下面的*参考答案*。

### 您的代码

In [ ]:
sentiment_prompts = [] # TODO: populate with sentiment analysis prompts for each statement in `statements`.
main_topic_prompts = [] # TODO: populate with main topic prompts for each statement in `statements`.
followup_prompts = [] # TODO: populate with followup question prompts for each statement in `statements`.

### 参考答案

In [ ]:
sentiment_prompts = [sentiment_template.invoke({"text": statement}) for statement in statements]

In [ ]:
main_topic_prompts = [main_topic_template.invoke({"text": statement}) for statement in statements]

In [ ]:
followup_prompts = [followup_template.invoke({"text": statement}) for statement in statements]

### 使用批处理为每个 LLM 任务生成响应

使用批处理一次性调用 LLM，针对您构建的每个提示词，分别处理每个任务。

如果您卡住了，可以随时查看下面的*参考答案*。

### 您的代码

In [ ]:
sentiments = [] # TODO: use the LLM to populate this list with the sentiment of each statement in `statements`.
main_topics = [] # TODO: use the LLM to populate this list with the main topic of each statement in `statements`.
followups = [] # TODO: use the LLM to populate this list with a followup question for each statement in `statements`.

### 参考答案

In [ ]:
sentiments = llm.batch(sentiment_prompts)

In [ ]:
main_topics = llm.batch(main_topic_prompts)

In [ ]:
followups = llm.batch(followup_prompts)

### 打印结果

最后，遍历原始语句和所有模型响应，给出一个完整的输出。

如果您卡住了，可以随时查看下面的*参考答案*。

### 您的代码

### 参考答案

In [ ]:
for statement, sentiment, main_topic, followup in zip(statements, sentiments, main_topics, followups):
    print(
        f"Statement: {statement}\n"
        f"Overall sentiment: {sentiment.content}\n"
        f"Main topic: {main_topic.content}\n"
        f"Followup question: {followup.content}\n"
    )

---

## 总结

通过将 LLM 功能提取到提示模板中，再批量调用聊天模型，您已经开始真正使用语言模型了。不难看出，您可以用至此所学的知识轻松进行进一步的扩展，以在大量文本数据上通过 LLM 进行文本分析和生成。

不过，我们才刚刚开始。下一个 notebook 将介绍一种叫作 LangChain 的表达语言（LCEL），这将使您能创建简洁而强大的 LLM 应用功能链。使用 LCEL 链将允许我们像在前面的练习中那样，更高效地构建各种功能。